In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, roc_auc_score

# Define paths
train_dir = 'path/to/reference/'  # Update this to your training directory path
test_dir = 'path/to/test/'        # Update this to your test directory path

# Define image dimensions
img_width, img_height = 64, 64

# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=10,
    class_mode='binary')

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Adjust this based on your dataset
    epochs=10  # Adjust the number of epochs as needed
)

# Evaluate the model
y_true = train_generator.classes
y_pred = model.predict(train_generator)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred.ravel()]

# Confusion Matrix and AUROC
conf_matrix = confusion_matrix(y_true, y_pred)
auroc = roc_auc_score(y_true, y_pred)

# Print confusion matrix and AUROC
print("Confusion Matrix:\n", conf_matrix)
print("AUROC:", auroc)

# Save model weights
model.save_weights('hw4.h5')

# Load model weights and predict on new data
model.load_weights('hw4.h5')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False)

predictions = model.predict(test_generator)
predictions = [1 if prob > 0.5 else 0 for prob in predictions.ravel()]

# Output predictions
for i, pred in enumerate(predictions):
    print(f'pic{i+1}.jpg - {"Dog" if pred == 1 else "Cat"}')
